In [ ]:
%cd ..
from pathlib import Path
import numpy as np
import torch
import imageio

import wandb
from src.models.nerf_pl import NeRF
from src.utils.generate_render_pose import c2w

In [ ]:
@torch.no_grad()
def render_imgs():
    with torch.device('cuda:1'):
        run = wandb.init(project='nerf')
        artifact = run.use_artifact("minjunsz/nerf/model-7w6f2b8s:v21", type="model") # type: ignore
        artifact_dir = artifact.download()
        nerf = NeRF.load_from_checkpoint(Path(artifact_dir) / "model.ckpt")
        nerf.eval()

        r, phi = 4.0, 50.0
        thetas = np.linspace(0, 360, 73)

        for theta in thetas:
            print(f"Theta {theta} is rendering now...")
            c2w_matrix = c2w(r, theta, phi)
            img = nerf.synthesis_novel_view(
                nerf.train_dataset.height,
                nerf.train_dataset.width,
                nerf.train_dataset.K,
                c2w_matrix,
            )
            wandb.log({"rendered": wandb.Image(img)})

render_imgs()

In [ ]:
api = wandb.Api()
run = api.run("minjunsz/nerf/d3ofaq39")

for file in run.files():
    if file.name.endswith('.png'):
        file.download()

gif_config = {
    'loop':0, ## 0으로 세팅하면 무한 반복, 3으로 설정하면 3번 반복
    'duration': 0.5 ## 다음 화면으로 넘어가는 시간
}

image_fnames = list(Path('./media/images').glob('*.png'))
image_fnames.sort(key=lambda x: int(x.name.split('_')[-2])) #sort by step
frames = [imageio.imread(image) for image in image_fnames]
imageio.mimwrite('./result.gif', ## 저장 경로
                frames, ## 이미지 리스트
                format='gif', ## 저장 포맷
                #**gif_config ## 부가 요소
            )